# ModelGridInterpolator

In practice, interaction with the model grid and bolometric correction objects is easiest through a `ModelGridInterpolator` object, which brings the two together.  This object is the replacement of the `Isochrone` object from previous generations of this package, though it has a slightly different API.  It is mostly backward compatible, except for the removal of the `.mag` function dictionary for interpolating apparent magnitudes, this being replaced by the `.interp_mag` method.

## Isochrones

In [ ]:
from isochrones.mist import MIST_Isochrone

mist = MIST_Isochrone()

In [ ]:
pars = [353, 9.78, -1.24]  # eep, log(age), feh
mist.interp_value(pars, ['mass', 'radius', 'Teff'])

In [ ]:
mist.interp_mag(pars + [200, 0.11], ['K', 'BP', 'RP'])  # Returns Teff, logg, feh, mags

## Evolution tracks

Note that you can do the same using an evolution track grid rather than an isochrone grid, though it takes different parameters:

In [ ]:
from isochrones.mist import MIST_EvolutionTrack

mist_track = MIST_EvolutionTrack()

In [ ]:
pars = [0.794, 353, -1.24]  # mass, eep, feh [matching above]
mist_track.interp_value(pars, ['mass', 'radius', 'Teff', 'age'])

In [ ]:
mist_track.interp_mag(pars + [200, 0.11], ['K', 'BP', 'RP'])

There are also convenience methods if you prefer (and for backward compatibility---note that the parameters must be unpacked, unlike the calls to `.interp_value` and `.interp_mag`), though it is slower to call multiple of these than to call `.interp_value` once with several desired outputs:

In [ ]:
mist_track.mass(*pars)

You can also get the dataframe of a single isochrone (interpolated to any age or metallicity) as follows:

In [ ]:
mist.isochrone(9.53, 0.1).head()  # just show first few rows

## Demo: Visualize

Now let's make sure that interpolated isochrones fall nicely between ones that are actually part of the grid.  In order to execute this code, you will need to 
    
    conda install -c pyviz pyviz
    
and to execute in JupyterLab, you will need to

    jupyter labextension install @pyviz/jupyterlab_pyviz


In [ ]:
import hvplot.pandas

iso1 = mist.model_grid.df.xs((9.5, 0.0), level=(0, 1))   # extract subgrid at log_age=9.5, feh=0.0
iso2 = mist.model_grid.df.xs((9.5, 0.25), level=(0, 1))  # extract subgrid at log_age=9.5, feh=0.25
iso3 = mist.isochrone(9.5, 0.12)  # should be between the other two

plot1 = iso1.hvplot.line('logTeff', 'logL', label='[Fe/H] = 0.0') 
plot2 = iso2.hvplot.line('logTeff', 'logL', label='[Fe/H] = 0.25') 
plot3 = iso3.hvplot.line('logTeff', 'logL', label='[Fe/H] = 0.12')

(plot1 * plot2 * plot3).options(invert_xaxis=True, legend_position='bottom_left')